# Exercício: Modelo de Linguagem com auto-atenção e máscaras causais

Seguimos na mesma linha de treinar um modelo de linguagem a partir dos textos do livro "O Guarani", de José de Alencar.

Neste exercício, vamos treinar um modelo de linguagem com auto-atenção e com máscara causal. A máscara causal é necessária para que o modelo não tenha acesso a palavras futuras, que é a abordagem usada por grandes modelos de linguagem, como o GPT.

Use a implementação matricial de auto-atenção da aula passada.

### Modificações necessárias

* Adicione a máscara causal na função `forward` da cabeça de auto-atenção.
* Modifique o nosso dataloader para retornar inputs (uma lista de tokens de tamanho $n$), targets (uma lista de tokens de tamanho $n$ deslocada para a esquerda em 1 token). Exemplo `input = [1, 2, 3, 4]`, `target = [2, 3, 4, 5]` para a sequência `[1, 2, 3, 4, 5]` com `seq_len=4`, por exemplo (Ver slide 50).

### Extra
* MultiHeadAttention: modifique a cabeça de auto-atenção para ter múltiplas cabeças. Isso não é obrigatório, mas pode ser interessante para ver como o modelo se comporta.
* Diagrama da geração: fazer diagrama que mostre os passos da geração de tokens (conforme slide 47).

### Dicas

* Use como base o vídeo do Karpathy: https://www.youtube.com/watch?v=kCc8FmEb1nY. Observe que, no vídeo, ele primeiro implementa um modelo bi-grama, depois um modelo de linguagem com auto-atenção. O modelo de auto-atenção é implementado por volta do minuto 40, mas vale a pena assistir o vídeo todo.
* Use esta implementação como base: https://colab.research.google.com/drive/1vFTg4MSXVJwNSzPjaCcvmqhxTP7gK7HA?usp=sharing. Observe como o modelo é organizado e como a máscara é implementada na classe MultiHeadAttention.
* Use `context_size=9`

------------------------

Observações: A resolução do exercício foi baseada nos notebooks implementados nas aulas anteriores e no vídeo do Karpathy.

## Parâmetros

In [1]:
# Livros (O Guarani)
urls = ["https://www.gutenberg.org/ebooks/67724.txt.utf-8", "https://www.gutenberg.org/ebooks/67725.txt.utf-8"]

# Dados do vocabulário
UNK = "<unk>"
vocab_size_desejado_sem_UNK = 10000 # Não considera o UNK
vocab_size = vocab_size_desejado_sem_UNK + 1

# Hiperparâmetros
# Esse conjunto faz o stride (step_inputs) de 1 em 1 na hora de gerar os dados de treinamento
# Entretanto, ele não usa train_test_split. Ele quebra o livro em um certo ponto e usa os 80% primeiros
# para treinamento e o resto de teste, de forma que não tenha muita sobreposição de texto
#
# PPL = 71 na época 3 (VOCAB = 3000)
# PPL = 145 na época 2 (VOCAB = 10000)
usar_train_test_split = False # Se True, usa train_test_split. Se False, separa o conjunto de treino e teste como uma posição do livro
context_size = 9  # número de palavras de entrada. O target é a próxima palavra
step_inputs = 1 # Se colocar < context_size e usar_train_test_split = True, vai ter sobreposição de sequências longas no treinamento e validação
num_epochs = 5
test_size = 0.2
batch_size=256
n_embd = 64
dropout = 0.2
n_head = 4
n_layer = 4
lr = 1e-3
seed = 18

# Usa train_test_split e considera o step_inputs = context_size. Isso vai diminuir o
# tamanho do conjunto de treinamento. É necessário reduzir o batch um pouco.
# PPL = 182 na época 7 (VOCAB = 10000)
# usar_train_test_split = True
# context_size = 9
# step_inputs = context_size
# num_epochs = 10
# test_size = 0.2
# batch_size=64
# n_embd = 64
# dropout = 0.2
# n_head = 4
# n_layer = 6
# lr = 1e-3
# seed = 18

## Faz download e carrega o dataset

Faz o download dos arquivos do projeto Gutenberg, extrai o conteúdo principal e armazena os parágrafos na variável `paragrafos` (lista). Em seguida, concatena os parágrafos com `\n` em uma única variável `livro`.

In [2]:
import requests

# Retorna True ou False dependendo se é um parágrafo válido ou não.
# Usado para remover parágrafos muito curtos ou outros casos específicos
# (por exemplo, os parágrafos do índice que tem vários pontos (.....))
def paragrafo_valido(paragrafo):
  return len(paragrafo) > 10 and '....' not in paragrafo and '***' not in paragrafo

# Dada uma URL do projeto Gutenberg, baixa o arquivo e divide o texto
# principal em parágrafos. Retorna um array de parágrafos
def carregar_paragrafos_livro(url, n_linhas_para_print=20):
  # Baixar o arquivo
  response = requests.get(url)
  texto = response.text

  # Encontrar o início e o fim do conteúdo principal do livro
  #inicio = texto.find("*** START OF THE PROJECT GUTENBERG EBOOK")
  str_inicio = "Ficão reservados os direitos de propriedade."
  str_fim = "*** END OF THE PROJECT GUTENBERG EBOOK"
  inicio = texto.find(str_inicio)
  fim = texto.find(str_fim)

  # Extrair o conteúdo principal do livro
  conteudo = texto[inicio+len(str_inicio):fim].replace('\r','')

  # Dividir o conteúdo em parágrafos e processar o conteúdo
  paragrafos = []

  # Cada parágrafo é separado por dois \n
  # Dentro de cada parágrafo, junta as linhas e remove espaços em branco no
  # início e fim do prágrafo.
  for paragrafo in conteudo.split("\n\n"):
    paragrafo = paragrafo.replace('\n', ' ').strip()
    if paragrafo_valido(paragrafo):
      paragrafos.append(paragrafo)

  # Imprime (usado pra debug)
  for p in paragrafos[0:n_linhas_para_print]:
    print(p)

  return paragrafos

paragrafos = []
for i, url in enumerate(urls, 1):
  print(f'-------------- Livro {i} ---------------')
  paragrafos.extend(carregar_paragrafos_livro(url))

print('-------------- -------------------------')
print(f'Total de parágrafos: {len(paragrafos)}')

-------------- Livro 1 ---------------
Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
Esta nova edição devia dar satisfação do empenho, que a extrema benevolencia do publico ledor, tão minguado ainda, mudou em bem para divida de reconhecimento.
Mais do que podia fiou de si o autor. Relendo a obra depois de annos, achou elle tão mau e incorrecto quando escrevera, que para bem corrigir, fora mister escrever de novo. Para tanto lhe carece o tempo e sobra o tedio de um labor ingrato.
Cingio-se pois ás pequenas emendas que toleravão o plano da obra e o desalinho de um estylo não castigado.
PRIMEIRA PARTE
OS AVENTUREIROS
De um dos cabeços da _Serra dos Órgãos_ deslisa um fio d'agua que se dirige para norte, e engrossado com os mananciaes, que recebe no seu curso de dez leguas, torna-se rio caudal.
É o _Paquequer_: soltando de cascata em cascata, enroscando-se como uma serpente, vai depois se espr

In [3]:
livro = "\n".join(paragrafos)

print(livro[0:1000])

Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
Esta nova edição devia dar satisfação do empenho, que a extrema benevolencia do publico ledor, tão minguado ainda, mudou em bem para divida de reconhecimento.
Mais do que podia fiou de si o autor. Relendo a obra depois de annos, achou elle tão mau e incorrecto quando escrevera, que para bem corrigir, fora mister escrever de novo. Para tanto lhe carece o tempo e sobra o tedio de um labor ingrato.
Cingio-se pois ás pequenas emendas que toleravão o plano da obra e o desalinho de um estylo não castigado.
PRIMEIRA PARTE
OS AVENTUREIROS
De um dos cabeços da _Serra dos Órgãos_ deslisa um fio d'agua que se dirige para norte, e engrossado com os mananciaes, que recebe no seu curso de dez leguas, torna-se rio caudal.
É o _Paquequer_: soltando de cascata em cascata, enroscando-se como uma serpente, vai depois se espreguiçar na varzea e embeber no Parahyba

## Análise do dataset

Primeiro define um tokenizador que vai separar as palavras para a entrada no dataset. A expressão regular deve capturar palavras, sinais de pontuação e quebras de linha. As quebras de linha são necessárias porque, ao final, quero que o modelo de linguagem também escreva parágrafos.

In [4]:
import re

def tokenizar(texto):
  texto = texto.lower()

  # Força os 3 pontos aparecerem juntos
  texto = texto.replace('...', ' SUBSTITUIRPORTRESPONTOS')

  # Define a expressão regular que captura palavras, sinais de pontuação e quebras de linha
  padrao = r'\b\w+\b|[^\w\s]|[\n]'

  # Usa o método findall para encontrar todas as ocorrências que se encaixam no padrão
  tokens = re.findall(padrao, texto)

  return tokens

print(tokenizar('Teste. Será que vai manter a pontuação?\nE os espaços em branco?\nDúvidas...'))

['teste', '.', 'será', 'que', 'vai', 'manter', 'a', 'pontuação', '?', '\n', 'e', 'os', 'espaços', 'em', 'branco', '?', '\n', 'dúvidas', 'SUBSTITUIRPORTRESPONTOS']


Contador de palavras:

In [5]:
# Conta as palavras no dataset
from collections import Counter

def count_words(text):
  word_counts = Counter()
  word_counts.update(tokenizar(text))
  return word_counts

word_counts = count_words(livro)

print('Total de tokens', len(word_counts))

top_tokens = word_counts.most_common(50)
for i, token_num_ocorrencia in enumerate(top_tokens, 1):
  print(f'{i}: {token_num_ocorrencia[0]} -> {token_num_ocorrencia[1]}')


Total de tokens 11870
1: , -> 7383
2: - -> 5932
3: . -> 5211
4: 
 -> 4575
5: a -> 4461
6: que -> 4336
7: o -> 4055
8: de -> 3951
9: e -> 3605
10: se -> 2398
11: ; -> 2355
12: um -> 1710
13: do -> 1404
14: não -> 1275
15: uma -> 1249
16: da -> 1133
17: os -> 1115
18: com -> 1015
19: sua -> 924
20: para -> 857
21: seu -> 777
22: ! -> 735
23: em -> 724
24: pery -> 718
25: as -> 702
26: no -> 640
27: por -> 621
28: ? -> 609
29: ao -> 592
30: como -> 587
31: lhe -> 558
32: d -> 491
33: á -> 490
34: tinha -> 478
35: era -> 462
36: cecilia -> 457
37: na -> 452
38: é -> 440
39: : -> 432
40: sobre -> 415
41: mas -> 410
42: elle -> 406
43: dos -> 373
44: indio -> 340
45: seus -> 324
46: me -> 324
47: mais -> 318
48: antonio -> 303
49: quando -> 288
50: SUBSTITUIRPORTRESPONTOS -> 281


## Criando um vocabulário

In [6]:
def gerar_vocabulario(texto, vocab_size_sem_UNK):
  counter = count_words(texto)

  # Considera apenas as palavras mais frequentes. Adiciona, na posição 0, o token UNK
  most_frequent_words = [UNK] + [word for word, count in counter.most_common(vocab_size_sem_UNK)]
  # vocab é um mapa de palavras para o índice correspondente. O mapa leva a palavra para um índice entre [0, vocab_size]
  # (o tamanho é vocab_size + 1), com o índice 0 apontando para UNK
  vocab = {word: i for i, word in enumerate(most_frequent_words)}

  return len(most_frequent_words), vocab, most_frequent_words

In [7]:
vocab_size, vocab, most_frequent_words = gerar_vocabulario(livro, vocab_size_desejado_sem_UNK)

print('Tamanho do vocabulário (considera UNK): ', vocab_size)

print('Posição 0: ', most_frequent_words[0])
print('Índice do UNK: ', vocab[UNK])
print('------------')
pos = 4 # a quarta maior ocorrência é o \n
print(f'Posição {pos}: ', most_frequent_words[pos])
print(f'Índice de {most_frequent_words[pos]}: ', vocab[most_frequent_words[pos]])

Tamanho do vocabulário (considera UNK):  10001
Posição 0:  <unk>
Índice do UNK:  0
------------
Posição 4:  

Índice de 
:  4


In [8]:
def encode_sentence(sentence, vocab):
  # Obs.: tem que usar o mesmo tokenizador que foi gerado o vocabulário
  return [vocab.get(word, 0) for word in tokenizar(sentence)]

def decode_sentence(sentence, most_frequent_words):
  words = [most_frequent_words[code] for code in sentence]
  decoded_sentence = ' '.join(words)
  decoded_sentence = decoded_sentence.replace('SUBSTITUIRPORTRESPONTOS', '...')
  decoded_sentence = decoded_sentence.replace(' \n ', '\n')
  return decoded_sentence

In [9]:
# Teste do encode/decode
frase = livro[0:200]

frase_encodada = encode_sentence(frase, vocab)
frase_reconstruida = decode_sentence(frase_encodada, most_frequent_words)

print('Original:')
print(frase)
print('Encodada:')
print(frase_encodada)
print('Reconstruída:')
print(frase_reconstruida)
print("--------------------------------------")

frase = "pery disse que amava cecilia"

frase_encodada = encode_sentence(frase, vocab)
frase_reconstruida = decode_sentence(frase_encodada, most_frequent_words)

print('Original:')
print(frase)
print('Encodada:')
print(frase_encodada)
print('Reconstruída:')
print(frase_reconstruida)


Original:
Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
Esta nova edição devia dar satisfação do empenho, que a 
Encodada:
[5872, 141, 4022, 23, 5873, 1, 10, 52, 111, 263, 264, 4023, 15, 981, 5874, 1, 6, 182, 131, 273, 7, 2040, 10, 5875, 5, 2434, 3, 4, 121, 1157, 4023, 173, 230, 917, 13, 5876, 1, 6, 5]
Reconstruída:
publicando este livro em 1857 , se disse ser aquella primeira edição uma prova typographica , que algum dia talvez o autor se dispuzesse a rever .
esta nova edição devia dar satisfação do empenho , que a
--------------------------------------
Original:
pery disse que amava cecilia
Encodada:
[24, 52, 6, 548, 36]
Reconstruída:
pery disse que amava cecilia


## Classe do dataset

Inicialmente, convertemos o livro para um array de tokens para, em seguida, gerar os conjuntos de treinamento e validação.

In [10]:
import torch

data = torch.tensor(encode_sentence(livro, vocab), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([139177]) torch.int64
tensor([5872,  141, 4022,   23, 5873,    1,   10,   52,  111,  263,  264, 4023,
          15,  981, 5874,    1,    6,  182,  131,  273,    7, 2040,   10, 5875,
           5, 2434,    3,    4,  121, 1157, 4023,  173,  230,  917,   13, 5876,
           1,    6,    5,  727, 3036,   13, 5877, 5878,    1,  108, 5879,   70,
           1, 3037,   23,  134,   20, 2435,    8,  918,    3,    4,   47,   13,
           6,  110, 5880,    8,  241,    7, 2040,    3, 5881,    5,  728,   64,
           8,  471,    1, 1068,   42,  108, 5882,    9, 5883,   49, 5884,    1,
           6,   20,  134, 5885,    1,  668, 5886, 4024,    8,  184,    3,   20,
         308,   31, 5887,    7])


Na construção do Dataset nos modelos de linguagem implementados nas aulas anteriores, cada frase de entrada (input) tinha apenas um target. Por exemplo, na frase "eu gosto de pizza", se a entrada for ["eu", "gosto", "de"], o target será "pizza".

No caso dos Transformers, o target também é um array. Na frase "eu gosto de pizza", se o input for ["eu", "gosto", "de"], o target será ["gosto", "de", "pizza"]. Isso ocorre pois os inputs são variáveis. O target i se refere a todos os inputs de 0 a i. Assim, quando a máscara causal é inserida, "gosto" se refere a ["eu"], "de" se refere a ["eu", "gosto"] e assim por diante.

A parte do janelamento continua. Dessa forma, em uma janela grande a gente tem algo mais ou menos assim:

Frase: ["eu", "gosto", "de", pizza", "de", "calabresa", "e", "portuguesa"]

input -> target

["eu", "gosto", "de"] -> [pizza", "de", "calabresa"]

["gosto", "de", "pizza"] -> ["de", "calabresa", "e"]

["de", "pizza", "de"] -> ["calabresa", "e", "portuguesa"]


Nota: No código abaixo, isso é gerado dessa forma se step_inputs = 1

In [11]:
def gera_inputs_e_targets_para_array(data: torch.Tensor, n):
  max = len(data)-n
  inputs = torch.stack([data[i:i+n] for i in range(0, max, step_inputs)])
  targets = torch.stack([data[i:i+n] for i in range(1, max+1, step_inputs)])

  return inputs, targets

In [12]:
# Exemplo de uso da função gera_inputs_e_targets_para_array
exemplo = encode_sentence("publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever", vocab)

inputs, targets = gera_inputs_e_targets_para_array(torch.tensor(exemplo), 3)

print('-------------------- decoded -------------------- ')
for input_target in zip(inputs, targets):
  print(f'{decode_sentence(input_target[0].tolist(), most_frequent_words)} -> {decode_sentence(input_target[1].tolist(), most_frequent_words)}')

print('-------------------- encoded -------------------- ')
for input_target in zip(inputs, targets):
  print(f'{input_target[0].tolist()} -> {input_target[1].tolist()}')

-------------------- decoded -------------------- 
publicando este livro -> este livro em
este livro em -> livro em 1857
livro em 1857 -> em 1857 ,
em 1857 , -> 1857 , se
1857 , se -> , se disse
, se disse -> se disse ser
se disse ser -> disse ser aquella
disse ser aquella -> ser aquella primeira
ser aquella primeira -> aquella primeira edição
aquella primeira edição -> primeira edição uma
primeira edição uma -> edição uma prova
edição uma prova -> uma prova typographica
uma prova typographica -> prova typographica ,
prova typographica , -> typographica , que
typographica , que -> , que algum
, que algum -> que algum dia
que algum dia -> algum dia talvez
algum dia talvez -> dia talvez o
dia talvez o -> talvez o autor
talvez o autor -> o autor se
o autor se -> autor se dispuzesse
autor se dispuzesse -> se dispuzesse a
se dispuzesse a -> dispuzesse a rever
-------------------- encoded -------------------- 
[5872, 141, 4022] -> [141, 4022, 23]
[141, 4022, 23] -> [4022, 23, 5873]
[4022, 23

In [13]:
from sklearn.model_selection import train_test_split

if usar_train_test_split:
  inputs, targets = gera_inputs_e_targets_para_array(data, context_size)
  inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=test_size, random_state=seed)
else:
  n = (int)((1-test_size)*len(data))
  inputs_train, targets_train = gera_inputs_e_targets_para_array(data[:n], context_size)
  inputs_val, targets_val = gera_inputs_e_targets_para_array(data[n:], context_size)

In [14]:
print(inputs_train[0])
print(targets_train[0])
print(inputs_train[1])
print(targets_train[1])

tensor([5872,  141, 4022,   23, 5873,    1,   10,   52,  111])
tensor([ 141, 4022,   23, 5873,    1,   10,   52,  111,  263])
tensor([ 141, 4022,   23, 5873,    1,   10,   52,  111,  263])
tensor([4022,   23, 5873,    1,   10,   52,  111,  263,  264])


In [15]:
%%time
from typing import Union
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GptDataset(Dataset):
  def __init__(self, inputs, targets):
    # Salva o vocabulário
    self.vocab = vocab

    # Agora vamos gerar os dados de treinamento e manter em cache
    self.inputs = inputs
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):
    return self.inputs[idx], self.targets[idx]

CPU times: user 3.35 ms, sys: 25 ms, total: 28.4 ms
Wall time: 90.2 ms


In [16]:
# Testa a implementação do Dataset

exemplo = "publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever"
inputs_ex, targets_ex = gera_inputs_e_targets_para_array(torch.tensor(encode_sentence(exemplo, vocab)), 3)
teste_dataset = GptDataset(inputs_ex, targets_ex)

print('Imprimindo o dataset')
for dados in teste_dataset:
  print(dados)

Imprimindo o dataset
(tensor([5872,  141, 4022]), tensor([ 141, 4022,   23]))
(tensor([ 141, 4022,   23]), tensor([4022,   23, 5873]))
(tensor([4022,   23, 5873]), tensor([  23, 5873,    1]))
(tensor([  23, 5873,    1]), tensor([5873,    1,   10]))
(tensor([5873,    1,   10]), tensor([ 1, 10, 52]))
(tensor([ 1, 10, 52]), tensor([ 10,  52, 111]))
(tensor([ 10,  52, 111]), tensor([ 52, 111, 263]))
(tensor([ 52, 111, 263]), tensor([111, 263, 264]))
(tensor([111, 263, 264]), tensor([ 263,  264, 4023]))
(tensor([ 263,  264, 4023]), tensor([ 264, 4023,   15]))
(tensor([ 264, 4023,   15]), tensor([4023,   15,  981]))
(tensor([4023,   15,  981]), tensor([  15,  981, 5874]))
(tensor([  15,  981, 5874]), tensor([ 981, 5874,    1]))
(tensor([ 981, 5874,    1]), tensor([5874,    1,    6]))
(tensor([5874,    1,    6]), tensor([  1,   6, 182]))
(tensor([  1,   6, 182]), tensor([  6, 182, 131]))
(tensor([  6, 182, 131]), tensor([182, 131, 273]))
(tensor([182, 131, 273]), tensor([131, 273,   7]))
(ten

### Instâncias vocabulário de treino, dataset e dataloader

Gera datasets e dataloader de treinamento e de teste. Para facilitar, vou continuar considerando que o vocabulário é o vocabulário do livro inteiro. Não vou gerar um vocabulário específico para o treinamento. Como o vocabulário é muito grande, não deve nem fazer diferença.

In [17]:
# Gera os dataset de treino e validação
train_dataset = GptDataset(inputs_train, targets_train)
val_dataset = GptDataset(inputs_val, targets_val)

In [18]:
print(f'len(inputs_train): {len(inputs_train)}')
print(f'len(inputs_val): {len(inputs_val)}')
print(f'Proporção de teste (deve ser próximo de {test_size}): {len(inputs_train)/(len(inputs_train)+len(inputs_val))}')

len(inputs_train): 111332
len(inputs_val): 27827
Proporção de teste (deve ser próximo de 0.2): 0.8000344929181727


In [19]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Model

In [20]:
# TODO O CÓDIGO DAQUI VEIO DO VÍDEO DO KARPATHY: https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy
import torch.nn as nn

class Head(nn.Module):
  def __init__(self, n_embd, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)

    # register_buffer fará com que tril seja um parâmetro não treinado pelo
    # modelo, mas persistente
    self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    # O que o Karpathy chama de B, T, C é:
    # B - batch size
    # T - context_size
    # C - head_size -> Para uma cabeça, o head_size costuma ser n_embd. Mas pode passar valor diferente também
    B, T, C = x.shape
    k = self.key(x)   # (B,T,C)
    q = self.query(x) # (B,T,C)

    # Multiplica q pela transposta de k.
    # A transposta é feita só nas duas últimas dimensões
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)

    # Isso daqui é a implementação da máscara causal
    # self.tril é uma matriz triangular de tamanho context_size (T):
    # 1 0 0 0
    # 1 1 0 0
    # 1 1 1 0
    # 1 1 1 1
    # O resultado de wei.masket_fill vai ser o seguinte:
    # x -inf -inf -inf
    # x   x  -inf -inf    => Onde x são os valores que já estavam em wei
    # x   x    x  -inf
    # x   x    x    x
    # E o softmax vai transformar isso nos pesos para calcular uma média.
    # Vamos supor que wei seja uma matriz de 1. Após a máscara, isso vai ficar:
    # 1 -inf -inf -inf
    # 1   1  -inf -inf
    # 1   1    1  -inf
    # 1   1    1    1
    # E, depois do softmax:
    # 1.00 0.00 0.00 0.00
    # 0.50 0.50 0.00 0.00
    # 0.33 0.33 0.33 0.00
    # 0.25 0.25 0.25 0.25
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # Calcula o v
    v = self.value(x) # (B,T,C)
    # O cálculo abaixo faz wei @ v, o que na prática calcula uma média
    # ponderada das linhas anteriores. Por exemplo, no caso de wei (q @ k.T)
    # ter gerado uma matriz de uns, wei, nesse ponto terá os pesos equivalentes
    # a uma média simples:
    # 1.00 0.00 0.00 0.00
    # 0.50 0.50 0.00 0.00
    # 0.33 0.33 0.33 0.00
    # 0.25 0.25 0.25 0.25
    #     @
    # 1  2  3  4
    # 5  6  7  8
    # 9  10 11 12
    # 13 14 15 16
    #
    # vira:
    # 1  2  3  4   -> média da linha 1
    # 3  4  5  6   -> média das linhas 1 e 2
    # 5  6  7  8   -> média das linhas 1, 2 e 3
    # 7  8  9  10  -> média das linhas 1, 2, 3 e 4
    out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)

    return out

class MultiHeadAttention(nn.Module):
  def __init__(self, n_embd, num_heads, head_size):
    super().__init__()
    # Define o multi head como um conjunto de heads
    self.heads = nn.ModuleList([Head(n_embd, head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([head(x) for head in self.heads], dim=-1)
    out = self.dropout(self.proj(out)) # A projeção WO é aplicada apenas no caso multihead
    return out

class FeedFoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    # Cria uma rede nos mesmos moldes do artigo do Bengio:
    # (linear + ReLU (no Bengio é tanh) + Linear)
    # e faz um dropout
    # No caso do Bengio, o tamanho da rede é um hiperparâmetro (h). Nesse caso,
    # é calculado como 4x o total de embeddings. Isso veio do artigo
    # "Attention is all you need", pag. 5: "The dimensionality of input and output
    # is d_model = 512, and the inner-layer has dimensionality dff = 2048"
    # No artigo, d_model é o total de embeddings
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_embd, n_head, head_size)
    self.ffwd = FeedFoward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    # É um bloco Transformer.
    # Começa com uma normalização da entrada (LayerNorm), passa para o
    # bloco de auto-atenção e adiciona a entrada (desvio na Fig. 1 do artigo
    # "Attention is all you need").
    x = x + self.sa(self.ln1(x))
    # Depois, normaliza de novo (LayerNorm), passa pelo bloco FeedForward e
    # diciona a entrada anterior (desvio na Fig. 1 do artigo
    # "Attention is all you need").
    x = x + self.ffwd(self.ln2(x))
    return x

class LanguageModel(nn.Module):
  def __init__(self, n_embd, context_size, n_head, n_layer, vocab_size):
    super().__init__()
    self.context_size = context_size

    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    # Nesse modelo os embeddings de posição também são aprendidos:
    self.position_embedding_table = nn.Embedding(context_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) # final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    # idx and targets are both (B,T) tensor of integers
    tok_emb = self.token_embedding_table(idx) # (B,T,C)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    x = tok_emb + pos_emb # (B,T,C)
    x = self.blocks(x) # (B,T,C)
    x = self.ln_f(x) # (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss = None
    else:
      # C, aqui, é o vocab_size.
      # logits tem o formato assim:
      # 0 0 0 .... 0.3 ... 0.7 ... 0
      #    ..... BT linhas .....
      # 1 0 0 ..................... 0
      # E targets, assim:
      # 78 ... 12 ... 32 (Total de BT elementos)
      #
      # F.cross_entropy vai verificar os índices do logits e comparar com o target.
      # Por exemplo, o primeiro target é 78. É desejado que o logito na posição 78 seja 1 e o resto 0
      # O último target é 32. Logo, é desejado que o logito na posição 32 seja 1 e o resto 0
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -self.context_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

## Training

Funções para calcular e imprimir a loss e a perplexidade de um DataLoader:

In [21]:
import math
from tqdm import tqdm

@torch.no_grad()
def calcula_loss_e_perplexidade(model, loader):
  model.eval()  # Coloca o modelo no modo de avaliação (não treinamento)
  loss = 0.0
  acc = 0
  for inputs, targets in tqdm(loader, desc='Calculando loss e perplexidade'):
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Forward pass
    logits, loss_batch = model(inputs, targets)
    # Acumula a perda
    loss += loss_batch*len(targets)
    acc += len(targets)

  loss /= acc
  ppl = math.exp(loss)

  model.train()

  return loss, ppl

def print_loss_ppl(msg, loss, ppl):
  print(f'{msg}. Loss: {loss:.2f}. Perplexidade: {ppl:.2f}\n')

Instancia um modelo e imprime o total de parâmetros, loss e perplexidade antes do treinamento:

In [22]:
# Como esse é o modelo que será treinado, seta a seed aqui
torch.manual_seed(seed)

model = LanguageModel(n_embd, context_size, n_head, n_layer, vocab_size)
model = model.to(device)

# print the number of parameters in the model
print('---------------- Total de parâmetros do modelo ----------------')
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

print(f'------------------ ANTES DE INICIAR O TREINAMENTO ------------------')
loss, ppl = calcula_loss_e_perplexidade(model, train_loader)
print_loss_ppl('\n[TRAIN]', loss, ppl)


---------------- Total de parâmetros do modelo ----------------
1.490001 M parameters
------------------ ANTES DE INICIAR O TREINAMENTO ------------------


Calculando loss e perplexidade: 100%|██████████| 435/435 [00:07<00:00, 54.86it/s]


[TRAIN]. Loss: 9.37. Perplexidade: 11678.98



Treinamento:

In [23]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

def treina_modelo(model, optimizer, train_loader, val_loader, num_epochs=num_epochs):
  for epoch in range(num_epochs):
    model.train()

    loss_train_epoch = 0
    acc = 0
    print(f'------------------ [ÉPOCA {epoch+1}/{num_epochs}] ------------------')
    for inputs, targets in tqdm(train_loader, desc='Treinando modelo'):
      inputs = inputs.to(device)
      targets = targets.to(device)

      logits, loss_batch = model(inputs, targets)
      optimizer.zero_grad()
      loss_batch.backward()
      optimizer.step()

      loss_train_epoch += loss_batch*len(targets)
      acc += len(targets)

    loss_train_epoch /= acc
    print_loss_ppl(f'[TRAIN Estimativa] Época {epoch+1}', loss_train_epoch, math.exp(loss_train_epoch))
    loss_val, ppl_val = calcula_loss_e_perplexidade(model, val_loader)
    print_loss_ppl(f'[EVAL] Época {epoch+1}', loss_val, ppl_val)

    # Salva os modelos intermediários
    checkpoint_path = f"modelo_epoca_{epoch+1}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)

In [24]:
treina_modelo(model, optimizer, train_loader, val_loader, num_epochs=num_epochs)

------------------ [ÉPOCA 1/5] ------------------


Treinando modelo: 100%|██████████| 435/435 [00:11<00:00, 37.13it/s]


[TRAIN Estimativa] Época 1. Loss: 5.71. Perplexidade: 302.27



Calculando loss e perplexidade: 100%|██████████| 109/109 [00:00<00:00, 113.93it/s]


[EVAL] Época 1. Loss: 5.11. Perplexidade: 165.35

------------------ [ÉPOCA 2/5] ------------------


...

[EVAL] Época 4. Loss: 5.15. Perplexidade: 172.54

------------------ [ÉPOCA 5/5] ------------------


Treinando modelo: 100%|██████████| 435/435 [00:11<00:00, 38.93it/s]


[TRAIN Estimativa] Época 5. Loss: 3.65. Perplexidade: 38.34



Calculando loss e perplexidade: 100%|██████████| 109/109 [00:00<00:00, 115.01it/s]


[EVAL] Época 5. Loss: 5.28. Perplexidade: 195.40



## Avaliação

In [25]:
def recupera_modelo(model, epoca):
  # Recupera o modelo salvo na época x
  checkpoint_path = f"modelo_epoca_{epoca}.pth"
  # Carregar o estado do checkpoint
  checkpoint = torch.load(checkpoint_path)
  # Aplicar o estado do modelo e otimizador carregados
  model.load_state_dict(checkpoint['model_state_dict'])

In [26]:
epoca_melhor_modelo = 3 if vocab_size_desejado_sem_UNK == 3000 else 2
recupera_modelo(model, epoca=epoca_melhor_modelo)

loss, ppl = calcula_loss_e_perplexidade(model, train_loader)
print_loss_ppl(f'\nLOSS e PPL de treinamento para o modelo da época {epoca_melhor_modelo}', loss, ppl)
loss, ppl = calcula_loss_e_perplexidade(model, val_loader)
print_loss_ppl(f'\nLOSS e PPL de validação para o modelo da época {epoca_melhor_modelo}', loss, ppl)

Calculando loss e perplexidade: 100%|██████████| 435/435 [00:03<00:00, 109.98it/s]



LOSS e PPL de treinamento para o modelo da época 2. Loss: 4.24. Perplexidade: 69.16



Calculando loss e perplexidade: 100%|██████████| 109/109 [00:00<00:00, 109.11it/s]


LOSS e PPL de validação para o modelo da época 2. Loss: 4.98. Perplexidade: 145.60



## Exemplo de uso

In [27]:
# Checa o final do livro (o final está no conjunto de validação)
print(livro[620000:623000])

 para elle a quem bastava o galho de uma arvore; mas para Cecilia.
Seguindo pela margem para escolher o lugar mais favoravel, Pery soltou uma palavra de surpreza vendo a canôa que se tinha embaraçado numa dessas ilhas fluctuantes feitas pelas parasitas do rio que boião sobre as aguas.
Era o melhor leito que podia ter a menina no meio do deserto; puxou a canôa, alcatifou o fundo com as folhas macias das palmeiras, e, tomando Cecilia nos braços, deitou-a no seu berço.
A menina não consentio que Pery remasse; e a canôa deslisou docemente pelo leito do rio, apenas impellida pela correnteza.
Cecilia brincava; debruçava-se sobre as aguas para colher uma flôr de passavem, para perseguir um peixe que beijava a face lisa das ondas, para ter o prazer de molhar as mãos nessa agua crystallina, para rever a sua imagem nesse espelho vacillante.
Quando tinha brincado bastante, voltava-se para seu amigo e fallava-lhe com o gazeio argentino, mimoso chilrear dos labios travessos de uma linda menina, ond

In [28]:
# Essa função pega um texto grande, pega apenas os primeiros context_size tokens dele e tenta gerar o resto
# Imprime o início e a continuação para comparar:
def continua_frase_e_printa(model, texto_entrada, n_tokens):
  # No código do Karpathy, a entrada tem tamanho (B, T)
  # Cria uma lista de tokens de entrada
  inicio_texto = ' '.join(tokenizar(texto_entrada)[:context_size])
  tokens_entrada = encode_sentence(inicio_texto, vocab)

  # Converte os tokens para tensor no tamanho (B=1, x). Se x > T, o generate trunca pra pegar só os últimos T tokens
  tensor_tokens_entrada = torch.tensor(tokens_entrada, dtype=torch.long, device=device).unsqueeze(0)

  print('-------------------------------')
  print(f'Texto para completar:\n\t{inicio_texto}\n')
  print(f'Texto gerado:\n\t{decode_sentence(model.generate(tensor_tokens_entrada, max_new_tokens=n_tokens)[0].tolist(), most_frequent_words)}\n')
  print(f'Texto do livro:\n\t{texto_entrada}')

continua_frase_e_printa(model, texto_entrada="Seguindo pela margem para escolher o lugar mais favoravel, Pery soltou uma palavra de surpreza vendo a canôa que se tinha embaraçado numa dessas ilhas fluctuantes feitas pelas parasitas do rio que boião sobre as aguas.", n_tokens=100)
continua_frase_e_printa(model, texto_entrada="Dahi a pouco os murmurios das aguas confundião-se com os accentos maviosos da voz de Cecilia que recitava o hymno christão repassado de tanta uncção e poesia.", n_tokens=100)
continua_frase_e_printa(model, texto_entrada="Logo que o sol chegou ao zenith, Pery procurou como na vespera um abrigo para passar as horas de calma.", n_tokens=100)
continua_frase_e_printa(model, texto_entrada="A canôa pojou n'um pequeno seio do rio, Cecilia saltou em terra; e seu companheiro escolheu uma sombra onde ella repousasse.", n_tokens=100)

-------------------------------
Texto para completar:
	seguindo pela margem para escolher o lugar mais favoravel

Texto gerado:
	seguindo pela margem para escolher o lugar mais favoravel , nas costas , e delirante ? quantas obrigado a conversa e <unk> . eu o immovel nunca : mandai quanto a figura de pery , onde tudo da mulher para sua prima : d . tua marcha sempre alvaro .
- - e eu furtar satisfazer , mas não me importa ; que tem o contacto , respondeu na sua existencia contra elles , e contrabando loredano dominou era mais melhor a parte do italiano ; quando a canto da cerca .
ergueu foi nisto aymorés e cauan negras a uma palavra de promessa do menina tinha

Texto do livro:
	Seguindo pela margem para escolher o lugar mais favoravel, Pery soltou uma palavra de surpreza vendo a canôa que se tinha embaraçado numa dessas ilhas fluctuantes feitas pelas parasitas do rio que boião sobre as aguas.
-------------------------------
Texto para completar:
	dahi a pouco os murmurios das aguas confu